In [ ]:
########################################################################
# Author    : A. Alsema
# Date      : Augustus 2021 
# Dataset   : Visium Spatial Transcriptomics for MS lesions, WM
# Purpose   : Make spatial module score plot.

# Required inputs:
# - 6.WM_integrated_scaled_subgroup_exclGM.rds: seurat object with subgroup info, 
# - markers: csv file with gene lists of interest, e.g. oligo-lineage_lists_for_modulescores.csv
# - outdir: directory for plots printed as pdf
# Output    : PDF with modulescore plots. Corresponding to figure 4a
#########################################################################

In [1]:
rm(list = ls())
library(Seurat)
library(ggplot2)
library(cowplot)
setwd("/data/bcn/Datasets/ST_AAL_MW_WM/")

outdir <- "Routput/Seurat/ModuleScores/spatialplots/"

Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode



In [2]:
#' Add a Small Legend to a ggplot. 
#' From https://stackoverflow.com/questions/52297978/decrease-overal-legend-size-elements-and-text
#'
#' This function modifies the legend of a ggplot object to make it smaller by adjusting the point size, text size, and space between legend items.
#'
#' @param myPlot A ggplot object to which the small legend will be added.
#' @param pointSize Numeric, specifying the size of the points in the legend. Default is 0.5.
#' @param textSize Numeric, specifying the size of the text in the legend. Default is 3.
#' @param spaceLegend Numeric, specifying the space between legend items, measured in lines. Default is 0.4.
#' 
#' @return A ggplot object with the modified, smaller legend.

#' @examples
#' library(ggplot2)
#' plot_small_legend <- addSmallLegend(plot)
#' print(plot_small_legend)
#' 
#' @export
addSmallLegend <- function(myPlot, pointSize = 0.5, textSize = 3, spaceLegend = 0.4) {
    myPlot = myPlot +
        guides(shape = guide_legend(override.aes = list(size = pointSize)),
               color = guide_legend(override.aes = list(size = pointSize))) +
        theme(legend.title = element_text(size = textSize), 
              legend.text  = element_text(size = textSize),
              legend.key.size = unit(spaceLegend, "lines"))
    return(myPlot)
}

## load data and compute module scores

In [3]:
datasets <- readRDS("RData/seurat/6.WM_integrated_scaled_subgroup_exclGM.rds")

DefaultAssay(datasets) <- "Spatial"

## option1: load gene lists from csv
markers <- read.csv(file = 'our_genesets/oligo-lineage_lists_for_modulescores.csv', header = T, stringsAsFactors = F)

## option2: manually define genes in markers
# markers <- c("CFAP299", "SPAG17", "ADGB", "CFAP43", "DNAH11", "DNAH9", "ZBBX")

markers <- as.data.frame(markers)

# calculate module scores and print plots

# prepare meaningful ggplot title
datasets$orig.ident = paste(datasets$sample_ID, datasets$Group, sep = "_")
samples <- names(datasets@images)
ST_ID <- unique(datasets$orig.ident)

meta.vars <- ncol(datasets@meta.data)

#calculate the module scores, can take 10min or so

for (column in 1:ncol(markers)){
    datasets <- AddModuleScore(datasets, features = list(na.omit(unique(markers[,column]))), name = colnames(markers)[column])
    }

# optional check, print head to see if the modules scores are calculated
head(datasets@meta.data)[meta.vars+1] 

# plot module score per sample inside the image.
datasets$orig.ident = paste(datasets$sample_ID, datasets$Group, sep = "_")
samples <- names(datasets@images)
ST_ID <- unique(datasets$orig.ident) # will be the title

# start is the first column index in datasets@meta.data to plot
start = meta.vars + 1
# loop over all gene sets, make a spatial feature plot per sample, place all the samples in a grid on 1 PDF page
plot_list = NULL
for (i in start:ncol(datasets@meta.data)){
    print(paste('plotting module scores', colnames(datasets@meta.data)[i]))
    for (k in 1:length(samples)){ # you don't need this if you have 1 sample
        sample <- as.character(samples[k])
        print(sample)
        plot <- SpatialFeaturePlot(datasets, images = sample, 
                                   image.alpha=0, # optional: deleting the HE image here
                                   stroke = NA,
                                   features = colnames(datasets@meta.data)[i],
                                   pt.size.factor = 1.4, # do not increase, spatial spots will overlap
                                   combine = TRUE)
        # add some custom layout
        plot <- addSmallLegend(plot)
        plot <- plot + 
                   ggtitle(as.character(ST_ID[k])) +
                   theme(plot.title = element_text(size = 9))
                   scale_colour_gradientn(colours = c("lightgrey", "lemonchiffon1", "lemonchiffon2","tan1", "firebrick2", "firebrick3", "magenta"))
        plot_list[[k]] <- plot
        }
    pdf(paste0(outdir, colnames(datasets@meta.data)[i], '.pdf'), width = 9, height = 9)
    #place all the samples in a grid on 1 PDF page
    print(plot_grid(plot_list[[1]], plot_list[[2]], plot_list[[4]], plot_list[[5]], plot_list[[6]], # CWM + NAWM
          plot_list[[15]], # inactive
          plot_list[[3]], plot_list[[18]], plot_list[[9]], plot_list[[10]], # active
          plot_list[[7]], # mixed
          plot_list[[8]], plot_list[[11]], plot_list[[12]], plot_list[[13]], plot_list[[14]], # mixed
          plot_list[[16]], plot_list[[17]], # remyl
                      ncol = 5))
    dev.off()
}